pip3 install SpeechRecognition pydub

https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python

In [1]:
import  re
import  pandas   as   pd
import  datetime

In [2]:
# a function that splits the audio file into chunks
# and applies speech recognition
def speech_2_text(path):
    
    # importing libraries 
    import speech_recognition as sr 
    import os 
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
    
    # create a speech recognition object
    r = sr.Recognizer()
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [3]:
filename =  "D:/Consulting/Matrimony/data/WhatsApp Ptt 2021-08-25 at 17.07.32.wav"
txt1     =  speech_2_text(filename)
print(txt1)

D:\Anaconda\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


audio-chunks\chunk1.wav : Profile for my son. 
audio-chunks\chunk2.wav : Bheem srinivasan. 
audio-chunks\chunk3.wav : Date of birth 2nd august 1985. 
audio-chunks\chunk4.wav : Religion hindu. 
audio-chunks\chunk5.wav : Masat ml. 
Profile for my son. Bheem srinivasan. Date of birth 2nd august 1985. Religion hindu. Masat ml. 


In [4]:
relationships_record =   'Myself|Daughter|Son|Sister|Brother|Relative|Friend'
relation             =   re.findall(relationships_record, txt1, flags = re.IGNORECASE)

In [5]:
word_split1          =   txt1.split('Profile for my son. Bheem')
word_split2          =   word_split1[1].split('Date of birth')
member_name          =   word_split2[0].strip() 
member_name 

'srinivasan.'

In [6]:
dob_part             =   word_split2[1].split('Religion')[0].strip()

In [7]:
dob                   =  dob_part.replace('nd', '')
dob                   =  dob.replace('.', '')
print(dob)

2 august 1985


In [8]:
dob1   =   datetime.datetime.strptime(dob,"%d %B %Y").strftime("%Y%m%d")
dob1

'19850802'

In [9]:
religion_record       =   'Hindu|Christian|Muslim|Others'
member_religion       =   re.findall(religion_record, word_split2[1], flags = re.IGNORECASE) 
member_religion

['hindu']

In [10]:
relation         =    ' '.join(map(str, relation))

In [11]:
relation      =   relation.lower()
if  relation == 'son':
    gender    =   'Male'
else:
    gender    =   'Female'    

In [12]:
my_df                =   pd.DataFrame({'Relationship' : relation, 'Name' : member_name , 'Gender' : gender,\
                                       'Date Of Birth' : dob1,\
                                      'Religion' : member_religion})

In [13]:
my_df.to_csv('Sample_matrimony_profile.csv', index = False)

### END